In [ ]:
import time
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
tushare_token = '604c146e2dc9044b250b3312916ef9e170734e53b0528384ccb736b7' 
pro = ts.pro_api(tushare_token)

# get daily stock data

## download daily stock quote

In [ ]:
date = pro.query('trade_cal', start_date='19991230', end_date='20200901')
trade_date = date[date['is_open']==1].reset_index(drop=True)
total = pd.DataFrame()
for i,date in enumerate(trade_date['cal_date'].tolist()[0:20]):
    daily_basic = pro.daily_basic(ts_code='', trade_date=date, fields='ts_code,trade_date,close,turnover_rate,volume_ratio,pe,pb,total_mv,circ_mv')
    #total = pd.concat([total,daily_basic],axis=0)
    daily_basic.to_csv('./stk_daily/'+date+'.csv',index=False)
    time.sleep(0.05)
    print(date,i)

## merge data

In [ ]:
import os
stk_list = os.listdir('./stk_daily/')
df_list = []
for stk in stk_list:
    temp = pd.read_csv('./stk_daily/'+stk)
    df_list.append(temp)

df_total = pd.concat(df_list)
df_total = df_total.sort_values(by=['trade_date','ts_code']).reset_index(drop=True)

## calculate stk ret

In [ ]:
# calculate stk ret
output = pd.DataFrame()
for stk,group in df_total.groupby('ts_code'):
    group['ret'] = group['close'].pct_change()
    output = pd.concat([output,group],axis=0)
    #print(stk)
output.sort_values(by=['trade_date','ts_code']).reset_index(drop=True).to_csv('stk_daily.csv',index=False)

# get monthly stock data

In [ ]:
date = pro.trade_cal(exchange='', start_date='20000101', end_date='20201231')
date = date[date['is_open']==1].reset_index(drop=True)
date['month'] = date['cal_date'].apply(lambda x:int( x[0:6] ))
date['cal_date'] = date['cal_date'].astype(int)
month_end = date.groupby('month').apply(lambda x:x.iloc[-1,:]).reset_index(drop=True)

In [ ]:
# monthly indicator data
indicator_list = []
for date in month_end['cal_date'].tolist():
    indicator = pro.query('daily_basic', ts_code='', trade_date=str(date),fields='ts_code,trade_date,close,total_mv,circ_mv,turnover_rate,volume_ratio,pe,pb')
    indicator_list.append(indicator)
indicator_df = pd.concat(indicator_list).sort_values(by=['trade_date','ts_code']).reset_index(drop=True)

# monthly ret data
Mret_list = []
for date in month_end['cal_date'].tolist():
    Mretdf = pro.monthly(trade_date=str(date), fields='ts_code,trade_date,pct_chg')
    Mret_list.append(Mretdf)
    print(date)
Mret = pd.concat(Mret_list)
pd.merge(indicator_df,Mret)#.to_csv('stk_monthly.csv',index=False)

# get ST stock list

In [ ]:
bs_code_list = list( map( lambda x: 'sh.'+x.split('.')[0] if x.split('.')[-1]=='SH' else 'sz.'+x.split('.')[0] , stk_month['ts_code'].unique().tolist()) )
# bs_date_list = pd.to_datetime(stk_month['trade_date'],format='%Y%m%d').dt.strftime(date_format='%Y-%m-%d')#
bs_date_list = list( map( lambda x:str(x)[0:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8],stk_month['trade_date'].unique().tolist() ))

In [ ]:
lg = bs.login()
total = []
for code in bs_code_list:
    rs = bs.query_history_k_data_plus(code,
        "date,code,tradestatus,pctChg,isST",
        start_date=bs_date_list[0], end_date=bs_date_list[-1],
        frequency="d", adjustflag="3")
    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    
    result = pd.DataFrame(data_list, columns=rs.fields)
    result = result[result['date'].isin(bs_date_list)].reset_index(drop=True)
    total.append(result)
    print(code)

In [ ]:
df = pd.concat(total,axis=0)
df.reset_index(drop=True).to_csv('st_file.csv',index=False)
st_df = pd.read_csv('st_file.csv')
st_df['ts_code'] = st_df['code'].apply(lambda x: x.split('.')[-1]+'.SZ' if x.split('.')[0]=='sz' else x.split('.')[-1]+'.SH')
st_df['trade_date'] = st_df['date'].apply(lambda x:int(x.replace('-','')))